In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Read file and stock into DataFrame

In [28]:
df = pd.read_csv("../Data/chessDB.txt", sep = "###", skiprows = [0, 1, 2, 3, 4], header=None, 
                 engine='python')

In [29]:
print(df.shape)
print(df.size)
df.head()

(3561470, 2)
7122940


,0,1
0,1 2000.03.14 1-0 2851 None 67 date_false resul...,W1.d4 B1.d5 W2.c4 B2.e6 W3.Nc3 B3.Nf6 W4.cxd5...
1,2 2000.03.14 1-0 2851 None 53 date_false resul...,W1.e4 B1.d5 W2.exd5 B2.Qxd5 W3.Nc3 B3.Qa5 W4....
2,3 1999.11.20 1-0 2851 None 57 date_false resul...,W1.e4 B1.e5 W2.Nf3 B2.Nc6 W3.Bc4 B3.Bc5 W4.c3...
3,4 1999.11.20 1-0 2851 None 49 date_false resul...,W1.e4 B1.d5 W2.exd5 B2.Qxd5 W3.Nc3 B3.Qa5 W4....
4,5 2000.02.20 1/2-1/2 2851 2633 97 date_false r...,W1.e4 B1.e5 W2.Nf3 B2.Nc6 W3.Bb5 B3.a6 W4.Ba4...


##### We need to split the df, to keep the moves as 1 field and split the field 0 into several fields

In [30]:
moves_df = df[[1]]

In [31]:
games_df = df[[0]]

In [32]:
games_df = games_df[0].str.split(" ", expand=True)

In [33]:
games_df = games_df.drop(columns=0)

In [34]:
moves_df.columns = ['moves']

In [35]:
games_df['moves'] = moves_df['moves']

In [36]:
df = games_df

##### Let's name the columns to make it easier to work with the data

In [37]:
df = df.drop(columns = 16) #empty row

In [38]:
columns = ['date', 'result', 'white_elo', 'black_elo', '# moves', 'date_corrupt', 'result_corrupt',
           'Welo_corrupt', 'Belo_corrupt', 'eventDate_corrupt', 'setup_corrupt', 'fen_corrupt', 
           'result2_corrupt', 'out_of_year range', 'game_length_corrupt', 'moves']

In [39]:
df.columns = columns

In [40]:
df.head(2)

,date,result,white_elo,black_elo,# moves,date_corrupt,result_corrupt,Welo_corrupt,Belo_corrupt,eventDate_corrupt,setup_corrupt,fen_corrupt,result2_corrupt,out_of_year range,game_length_corrupt,moves
0,2000.03.14,1-0,2851,None,67,date_false,result_false,welo_false,belo_true,edate_true,setup_false,fen_false,result2_false,oyrange_false,blen_false,W1.d4 B1.d5 W2.c4 B2.e6 W3.Nc3 B3.Nf6 W4.cxd5...
1,2000.03.14,1-0,2851,None,53,date_false,result_false,welo_false,belo_true,edate_true,setup_false,fen_false,result2_false,oyrange_false,blen_false,W1.e4 B1.d5 W2.exd5 B2.Qxd5 W3.Nc3 B3.Qa5 W4....


• date: date of the game <br> <br>
• result: 1-0 if white won, 0-1 if black won, 1/2-1/2 if draw and * if missing <br> <br>
• white_elo: the elo (strength) of the white player; the higher, the better the player <br> <br>
• black_elo: the elo (strength) of the black player; the higher, the better the player <br> <br>
• # moves: number of moves in the game <br> <br>
• date_corrupt: date_true if the date is corrupted in some way, date_false otherwise <br> <br>
• result_corrupt: result_true if the result of the game is somehow corrupted, result_false otherwise <br> <br>
• Welo_corrupt: welo_true if the white elo is corrupted, welo_false if not <br> <br>
• Belo_corrupt: belo_true if the black elo is corrupted, belo_false if not <br> <br>
• evenDate_corrupt: edate_true if the date of the event where the game was held id corrupted or not, edate_false otherwise <br> <br>
• setup_corrupt: setup_true if the game played was a Fischer Random Chess or the initial setup is somehow corrupted, setup_false otherwise <br> <br>
• fen_corrupt: fen_true or fen_false <br> <br>
• result2_corrupt: In the original file the result is provided in two places. At the end of each sequence of moves and in the attributes part. This flag indicates if the result is (is not) properly provided after the sequence of moves (just for checking consistency in the PGN file). <br> <br>
• out_of_year range: oyrange_false if the game was held in the range [1998, 2007].  <br> <br>
• game_length_corrupt: blen_true if the length of the game is not good, blen_false otherwise <br> <br>

## Remove irrelevant rows and columns

In [41]:
clean_df = df.copy(deep=True)

In [42]:
print(df[df.game_length_corrupt == 'blen_true']['moves'].unique())
clean_df = clean_df[clean_df.game_length_corrupt == 'blen_false']
# nothing but incorrect games, we remove them
# we also remove the row 'game_length_corrupt' as it is now irrelevant
clean_df = clean_df.drop(columns = 'game_length_corrupt')

[nan]


In [43]:
# we don't care about the date of the game
clean_df = clean_df.drop(columns = ['date', 'date_corrupt', 'eventDate_corrupt', 'out_of_year range'])

In [44]:
len(clean_df[clean_df.fen_corrupt == 'fen_true'])

817

In [45]:
# the games where fen_corrupt is true do not correspond to possible games, the moves registered have errors
# thus we remove theses games (there are only 817 of them)
clean_df = clean_df[clean_df.fen_corrupt == 'fen_false']
# we may now remove the column 'fen_corrupt' as it is now always equal to fen_true
clean_df = clean_df.drop(columns = 'fen_corrupt')

In [46]:
# we check that when result_corrupt is true, result2_corrupt is also true
# thus we can only analyse one of the two
print(clean_df[clean_df.result_corrupt == 'result_true'].result2_corrupt.unique())

# we can also see that there are only 13 games where the result is corrupted
nb = len(clean_df[clean_df.result_corrupt == 'result_true'])
print(f"There are {nb} games where the result is corrupted")
# hence, we just get rid of them
clean_df = clean_df[clean_df.result_corrupt == 'result_false']
clean_df = clean_df.drop(columns = ['result_corrupt', 'result2_corrupt'])

['result2_true']
There are 10 games where the result is corrupted


In [47]:
# we don't really care whether the elo of either player is missing or not
clean_df = clean_df.drop(columns = ['Welo_corrupt', 'Belo_corrupt'])

In [48]:
clean_df.columns

Index(['result', 'white_elo', 'black_elo', '# moves', 'setup_corrupt',
       'moves'],
      dtype='object')

In [49]:
clean_df['# moves'] = clean_df['# moves'].astype('int')

In [50]:
print(clean_df.setup_corrupt.unique())
#turns out that, after cleaning our dataset, there no row in clean_df where
#setup_corrupt is True, thus we can remove this field
clean_df = clean_df.drop(columns = 'setup_corrupt')

['setup_false']


In [51]:
#we used to have a mixed type for the elos, with intergers and string if missing
#we fix it, and put an negative elo (-1) if the elo happens to be missing
clean_df['white_elo'] = clean_df['white_elo'].replace('None', -1)
clean_df['black_elo'] = clean_df['black_elo'].replace('None', -1)

In [52]:
# let's also remove the games where the numbher of moves is quite low
# there aren't too many games like that
len(clean_df[clean_df["# moves"] <= 10])
clean_df = clean_df.loc[clean_df["# moves"] > 10]

In [53]:
clean_df

,result,white_elo,black_elo,# moves,moves
0,1-0,2851,-1,67,W1.d4 B1.d5 W2.c4 B2.e6 W3.Nc3 B3.Nf6 W4.cxd5...
1,1-0,2851,-1,53,W1.e4 B1.d5 W2.exd5 B2.Qxd5 W3.Nc3 B3.Qa5 W4....
2,1-0,2851,-1,57,W1.e4 B1.e5 W2.Nf3 B2.Nc6 W3.Bc4 B3.Bc5 W4.c3...
3,1-0,2851,-1,49,W1.e4 B1.d5 W2.exd5 B2.Qxd5 W3.Nc3 B3.Qa5 W4....
4,1/2-1/2,2851,2633,97,W1.e4 B1.e5 W2.Nf3 B2.Nc6 W3.Bb5 B3.a6 W4.Ba4...
...,...,...,...,...,...
3561465,1-0,-1,-1,57,W1.e4 B1.e6 W2.d4 B2.d5 W3.e5 B3.c5 W4.dxc5 B...
3561466,1/2-1/2,-1,-1,80,W1.d4 B1.Nf6 W2.Nc3 B2.g6 W3.Bg5 B3.Bg7 W4.Nf...
3561467,1-0,-1,-1,57,W1.c4 B1.Nf6 W2.Nc3 B2.g6 W3.e4 B3.d6 W4.d4 B...
3561468,1-0,-1,-1,80,W1.g3 B1.d5 W2.Bg2 B2.c5 W3.d3 B3.Nf6 W4.Nf3 ...


In [29]:
clean_df.to_csv("../Data/clean_df.csv", index=False)